In [18]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [19]:
with open('intent.json') as file:
    data = json.load(file)

In [20]:
training_sentences = []
training_labels = []
labels = []
responses = []

In [21]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        

num_classes = len(labels)

In [22]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [23]:
training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

In [24]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [25]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

d:\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [26]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.1531 - loss: 2.0803
Epoch 2/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2449 - loss: 2.0777
Epoch 3/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.1837 - loss: 2.0765  
Epoch 4/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.2143 - loss: 2.0757  
Epoch 5/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3062 - loss: 2.0748 
Epoch 6/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.2652 - loss: 2.0737  
Epoch 7/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2756 - loss: 2.0723
Epoch 8/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.2039 - loss: 2.0718  
Epoch 9/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.1531 - loss: 2.0703  
Epoch 10/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.1427 - loss: 2.0706  
Epoch 11/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.1531 - loss: 2.0701  
Epoch 12/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.1531 - loss

In [27]:
# to save the trained model
model.save("chat_model.keras")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import tensorflow as tf
import pickle
import numpy as np

# Initialize the Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Load the trained model
model = tf.keras.models.load_model('chat_model.keras')

# Load the tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load the label encoder
with open('label_encoder.pickle', 'rb') as ecn_file:
    lbl_encoder = pickle.load(ecn_file)

@app.route('/api/chatbot', methods=['POST'])
def chatbot():
    user_message = request.json.get('message')
    
    # Preprocess the user message
    sequences = tokenizer.texts_to_sequences([user_message])
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=20)

    # Predict the intent
    prediction = model.predict(padded_sequences)
    predicted_label = np.argmax(prediction)
    intent = lbl_encoder.inverse_transform([predicted_label])[0]

    response_text = f'The predicted intent is: {intent}'

    return jsonify({'text': response_text})

if __name__ == '__main__':
    app.run(port=3000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:3000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

d:\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import gradio as gr

In [ ]:
# import json 
# import numpy as np
# from tensorflow import keras
# from sklearn.preprocessing import LabelEncoder

# import colorama 
# colorama.init()
# from colorama import Fore, Style, Back

# import random
# import pickle

# with open("intent.json") as file:
#     data = json.load(file)


# def chat():
#     # load trained model
#     model = keras.models.load_model('chat_model')

#     # load tokenizer object
#     with open('tokenizer.pickle', 'rb') as handle:
#         tokenizer = pickle.load(handle)

#     # load label encoder object
#     with open('label_encoder.pickle', 'rb') as enc:
#         lbl_encoder = pickle.load(enc)

#     # parameters
#     max_len = 20
    
#     while True:
#         print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
#         inp = input()
#         if inp.lower() == "quit":
#             break

#         result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
#                                              truncating='post', maxlen=max_len))
#         tag = lbl_encoder.inverse_transform([np.argmax(result)])

#         for i in data['intents']:
#             if i['tag'] == tag:
#                 print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

#         # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

# print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
# # chat()

# iface = gr.Interface(fn=chat, inputs= "text", outputs="text", title="My Chatbot")
# iface.launch()

In [ ]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import colorama 
colorama.init()
from colorama import Fore, Style, Back
import random
import pickle
import gradio as gr

# Load data and model
with open("intent.json") as file:
    data = json.load(file)

model = keras.models.load_model('chat_model.keras')
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
with open('label_encoder.pickle', 'rb') as enc:
    lbl_encoder = pickle.load(enc)

max_len = 20

# Define function for chatbot
def chat(query):
    result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([query]),
                                         truncating='post', maxlen=max_len))
    tag = lbl_encoder.inverse_transform([np.argmax(result)])

    for i in data['intents']:
        if i['tag'] == tag:
            response = np.random.choice(i['responses'])
            return response

iface = gr.Interface(fn=chat, inputs="text", outputs="text", title="My Chatbot")
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7862

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
